In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [6]:
diamonds_train = pd.read_csv('outputs/df_train.csv')
diamonds_train_price = pd.read_csv('outputs/df_train_price.csv')
diamonds_predict=pd.read_csv('outputs/df_test.csv')
submit=pd.read_csv('inputs/sample_submission.csv')

In [7]:
diamonds_train.head()

,carat,cut_numeric,color_numeric,clarity_numeric,depth,table,x,y,z
0,-1.051355,-0.810564,0.352161,-0.033632,0.736126,-0.650176,-1.283716,-1.236438,-1.180531
1,-0.966764,0.982588,0.940769,-0.641579,0.596363,-1.097476,-1.132102,-1.080061,-1.039596
2,-0.839879,-0.810564,1.529377,-0.641579,-1.010919,2.033618,-0.918059,-0.854183,-0.969129
3,-0.839879,0.086012,-0.825055,0.574315,0.037308,0.781181,-0.900222,-0.862870,-0.870475
4,0.217504,-0.810564,1.529377,-0.641579,-0.521746,2.480917,0.330526,0.344710,0.271096


In [8]:
diamonds_predict.head()

,carat,cut_numeric,color_numeric,clarity_numeric,depth,table,x,y,z
0,3.267015,0.978152,-1.416151,-1.232414,-0.666318,-1.554540,2.620734,2.539800,2.446736
1,2.596725,0.978152,-0.830504,-1.232414,0.168508,-0.658049,2.175567,2.234832,2.230593
2,-0.608098,0.978152,-1.416151,-0.627731,-0.040199,-1.554540,-0.495431,-0.491942,-0.492811
3,-1.047976,0.978152,-1.416151,-0.627731,-0.318474,-0.658049,-1.261117,-1.263332,-1.285335
4,0.334497,-2.592365,-0.830504,-0.023048,4.899190,-0.658049,0.332578,0.216660,0.890505


# Split train and test

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diamonds_train, diamonds_train_price, test_size=0.01)

In [10]:
X_train.head()

,carat,cut_numeric,color_numeric,clarity_numeric,depth,table,x,y,z
31052,-0.522664,0.086012,0.940769,-0.033632,-2.548318,2.033618,-0.240255,-0.306862,-0.546325
35986,0.873081,0.982588,0.940769,0.574315,-0.032574,-0.650176,0.972656,0.970219,0.961676
37647,0.217504,0.982588,-1.413663,-0.033632,0.177072,-1.992074,0.392956,0.431586,0.440218
21214,-0.818731,0.982588,0.352161,1.790210,0.037308,-0.202877,-0.873466,-0.819432,-0.828194
12204,-0.459221,0.982588,-0.236447,1.790210,-0.312101,-0.650176,-0.329440,-0.289487,-0.334923


In [7]:
y_test.head()

,price
22844,2943
4842,1875
39809,1939
18847,473
20662,3462


# Apache Spark  Random forest
https://spark.apache.org/docs/2.1.0/ml-classification-regression.html#random-forest-classifier<br>
https://docs.microsoft.com/it-it/azure/hdinsight/spark/apache-spark-machine-learning-mllib-ipython<br>
https://www.bmc.com/blogs/python-spark-machine-learning-classification/<br>
https://databricks.com/spark/getting-started-with-apache-spark/machine-learning<br>

In [1]:
#pyspark --num-executors 5 --driver-memory 8g --executor-memory 8g

In [2]:
#clf = ensemble.RandomForestClassifier(verbose=3) verbose para saber el tiempo de procesamiento
from pyspark import SparkContext
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer

In [12]:
diamonds_train['price']= diamonds_train_price

In [13]:
spark_df = diamonds_train[['carat', 'cut_numeric','color_numeric','clarity_numeric','depth','table','x','y','z', 'price']]

In [14]:
spark_df = spark.createDataFrame(spark_df)

In [15]:
spark_df.show()

+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-----+
|               carat|        cut_numeric|       color_numeric|     clarity_numeric|               depth|               table|                   x|                  y|                  z|price|
+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-----+
|  -1.051354945258851|-0.8105638153544471| 0.35216095194517794|-0.03363208651838169|  0.7361262077726191| -0.6501764299800551| -1.2837156287583358|-1.2364383442799414| -1.180530943376829|  605|
| -0.9667643697664698| 0.9825875572142716|   0.940769087416264| -0.6415793877253191|  0.5963626245318632| -1.0974755411742854|  -1.132101739428727|-1.0800610391334775|-1.0395962653514643|  565|
| -0.8398785065278978|-0.81056

In [16]:
spark_df.registerTempTable("diamond_spark")

In [17]:
from pyspark.ml.feature import VectorAssembler

In [18]:
assembler = VectorAssembler(
    inputCols=['carat', 'cut_numeric','color_numeric','clarity_numeric','depth','table','x','y','z'],
    outputCol="features")
output = assembler.transform(spark_df)

In [19]:
output.show()

+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-----+--------------------+
|               carat|        cut_numeric|       color_numeric|     clarity_numeric|               depth|               table|                   x|                  y|                  z|price|            features|
+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-----+--------------------+
|  -1.051354945258851|-0.8105638153544471| 0.35216095194517794|-0.03363208651838169|  0.7361262077726191| -0.6501764299800551| -1.2837156287583358|-1.2364383442799414| -1.180530943376829|  605|[-1.0513549452588...|
| -0.9667643697664698| 0.9825875572142716|   0.940769087416264| -0.6415793877253191|  0.5963626245318632| -1.0974755411742854|  -1.132101739

In [20]:
output.registerTempTable("output")

In [21]:
#val Array(trainingData, testData) = df_spark.randomSplit(Array(0.8, 0.2)).setImpurity("gini").setMaxDepth(3).setNumTrees(20).setFeatureSubsetStrategy("auto")

In [22]:
data_spark = spark.sql("""
SELECT features,price
FROM output""")

In [23]:
data_spark.show()

+--------------------+-----+
|            features|price|
+--------------------+-----+
|[-1.0513549452588...|  605|
|[-0.9667643697664...|  565|
|[-0.8398785065278...|  720|
|[-0.8398785065278...|  793|
|[0.21750368712686...| 4381|
|[-0.4169256290659...| 2263|
|[-1.0090596575126...|  524|
|[-1.0513549452588...|  844|
|[-0.5861067800507...| 1577|
|[-0.8187308626548...|  705|
|[-1.0302073013857...|  429|
|[-1.0302073013857...|  625|
|[-0.2054491903350...| 1865|
|[-0.0997109709695...| 2216|
|[0.72504714008115...| 7079|
|[0.00602724839591...| 3460|
|[2.56489215704045...|14725|
|[0.87308064719282...| 8950|
|[0.21750368712686...| 2873|
|[1.50750996338568...|14889|
+--------------------+-----+
only showing top 20 rows



In [24]:
data_spark = data_spark.withColumnRenamed("price","label")

In [25]:
data_spark.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[-1.0513549452588...|  605|
|[-0.9667643697664...|  565|
|[-0.8398785065278...|  720|
|[-0.8398785065278...|  793|
|[0.21750368712686...| 4381|
|[-0.4169256290659...| 2263|
|[-1.0090596575126...|  524|
|[-1.0513549452588...|  844|
|[-0.5861067800507...| 1577|
|[-0.8187308626548...|  705|
|[-1.0302073013857...|  429|
|[-1.0302073013857...|  625|
|[-0.2054491903350...| 1865|
|[-0.0997109709695...| 2216|
|[0.72504714008115...| 7079|
|[0.00602724839591...| 3460|
|[2.56489215704045...|14725|
|[0.87308064719282...| 8950|
|[0.21750368712686...| 2873|
|[1.50750996338568...|14889|
+--------------------+-----+
only showing top 20 rows



In [26]:
'''train_y_spark = spark.sql("""
SELECT price
FROM trainingData""")'''

'train_y_spark = spark.sql("""\nSELECT price\nFROM trainingData""")'

In [27]:
#testData.registerTempTable("testData")

In [28]:
'''testData = spark.sql("""
SELECT features, price
FROM testData""")
testData.show()'''

'testData = spark.sql("""\nSELECT features, price\nFROM testData""")\ntestData.show()'

In [29]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data_spark)

# Automatically identify categorical features, and index them.

# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(data_spark)

In [30]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data_spark.randomSplit([0.8, 0.2])

In [31]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

In [32]:
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="label", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [33]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

In [34]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

Py4JJavaError: An error occurred while calling o116.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 14.0 failed 1 times, most recent failure: Lost task 4.0 in stage 14.0 (TID 66, LISCIA-OMEN-by-HP-Laptop.lan, executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.ml.tree.impl.DTStatsAggregator.<init>(DTStatsAggregator.scala:77)
	at org.apache.spark.ml.tree.impl.RandomForest$.$anonfun$findBestSplits$22(RandomForest.scala:651)
	at org.apache.spark.ml.tree.impl.RandomForest$.$anonfun$findBestSplits$22$adapted(RandomForest.scala:647)
	at org.apache.spark.ml.tree.impl.RandomForest$$$Lambda$3419/0x0000000101505840.apply(Unknown Source)
	at scala.Array$.tabulate(Array.scala:334)
	at org.apache.spark.ml.tree.impl.RandomForest$.$anonfun$findBestSplits$21(RandomForest.scala:647)
	at org.apache.spark.ml.tree.impl.RandomForest$$$Lambda$3392/0x00000001014be040.apply(Unknown Source)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartiti

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 51072)
Traceback (most recent call last):
  File "/home/must4in3/anaconda3/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/must4in3/anaconda3/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/must4in3/anaconda3/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/must4in3/anaconda3/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/must4in3/spark/python/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/home/must4in3/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/home/must4in3/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num

In [ ]:
# Make predictions.
predictions_rf = model.transform(testData)

In [ ]:
# Select example rows to display.
predictions_rf.select("predictedLabel", "label", "features").show(5)

In [ ]:
df = predictions_rf.select("predictedLabel", "label")

In [ ]:
df.toPandas()


In [ ]:
df.select("predictedLabel", "label").toPandas()

In [ ]:
# Select (prediction, true label) and compute test error
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  

In [ ]:
predictions.show()

# Prueba

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
data_spark = data_spark.withColumnRenamed("price","label")

In [ ]:
data_spark.show()

In [ ]:
modelA = lr.fit(data_spark, {lr.regParam:0.0})
modelB = lr.fit(data_spark, {lr.regParam:100.0})

In [ ]:
predictionsA = modelA.transform(data_spark)
predictionsA.show()

In [ ]:
result_pdf = predictionsA.select("*").toPandas()

In [ ]:
result_pdf

In [ ]:
print( "Errore quadratico medio:% .2f" % mean_squared_error ( result_pdf['label'], result_pdf['prediction']))
print( 'R²:% .2f' % r2_score ( result_pdf['label'], result_pdf['prediction']))

# Random Forest without spark


In [ ]:
features = pd.get_dummies(X_train)

In [ ]:
features

In [ ]:
features.to_numpy()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=100, max_depth=40, 
                                 n_jobs=-1, verbose=3 )

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred_rf = model.predict(X_test)

In [ ]:
df_y_rf = pd.DataFrame(y_pred_rf)

In [ ]:
df_y_rf

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced', 
                                 solver ='saga' , n_jobs=-1, verbose=3 )

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
df_ylogistic = pd.DataFrame(y_pred)

In [ ]:
df_ylogistic

In [ ]:
y_test_logistic = y_test.copy()
y_test_logistic['y']= y_pred

In [ ]:
y_test_logistic['Price-y'] = abs(y_test_logistic['price']- y_test_logistic['y'])
y_test_logistic.describe()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
print( "Errore quadratico medio:% .2f" % mean_squared_error ( y_test_logistic['price'], y_test_logistic['y']))
print( 'R²:% .2f' % r2_score ( y_test_logistic['price'], y_test_logistic['y']))